In [1]:
import os
os.environ['PROJECT_PATH'] = os.path.abspath(os.curdir)

**Mount Google Drive** 
Tokenization will not work due to Perl script usage

In [ ]:
# from google.colab import drive
# os.environ['PROJECT_PATH']='/content/ydrive/My Drive/Study/DS_Project'
# drive.mount('/content/ydrive/')

In [2]:
os.environ['TOOLS']='%s/tools' % os.environ['PROJECT_PATH']
os.environ['RESOURCES']='%s/resources' % os.environ['PROJECT_PATH']
os.environ['DATA']='%s/data' % os.environ['RESOURCES']

## Preprocessing

### Download data

In [5]:
 ![ -f "$RESOURCES/data/news.2017.cs.shuffled" ] || wget http://data.statmt.org/wmt18/translation-task/news.2017.cs.shuffled.deduped.gz -P "$RESOURCES/data"
![ -f "$RESOURCES/data/news.2017.en.shuffled" ] || wget http://data.statmt.org/wmt18/translation-task/news.2017.en.shuffled.deduped.gz -P "$RESOURCES/data"
!gzip -d "$RESOURCES/data/news.2017.cs.shuffled.deduped.gz"
!gzip -d "$RESOURCES/data/news.2017.en.shuffled.deduped.gz"
os.environ['L1']='cs'
os.environ['L2']='en'
os.environ['L1_DATA']="news.2017.cs.shuffled"  
os.environ['L2_DATA']="news.2017.en.shuffled"

--2020-03-14 18:47:07--  http://data.statmt.org/wmt18/translation-task/news.2017.cs.shuffled.deduped.gz
Resolving data.statmt.org (data.statmt.org)...129.215.197.184
Connecting to data.statmt.org (data.statmt.org)|129.215.197.184|:80...connected.
HTTP request sent, awaiting response...200 OK
Length: 337728864 (322M) [application/x-gzip]
Saving to: ‘/home/hawk/Programming/Study/DS_Project/DS_Project/resources/data/news.2017.cs.shuffled.deduped.gz’

                      5%[>                   ]  16,42M  1,87MB/s    eta 2m 48s^C
--2020-03-14 18:47:17--  http://data.statmt.org/wmt18/translation-task/news.2017.en.shuffled.deduped.gz
Resolving data.statmt.org (data.statmt.org)... 129.215.197.184
Connecting to data.statmt.org (data.statmt.org)|129.215.197.184|:80... connected.
HTTP request sent, awaiting response...200 OK
Length: 1387271195 (1,3G) [application/x-gzip]
Saving to: ‘/home/hawk/Programming/Study/DS_Project/DS_Project/resources/data/news.2017.en.shuffled.deduped.gz’

            

### Text cleaning and tokenization
[Source](https://github.com/facebookresearch/XLM/blob/master/tools/tokenize.sh)

In [4]:
%%bash
MOSES="$TOOLS/mosesdecoder"
git clone https://github.com/moses-smt/mosesdecoder "$MOSES"

REPLACE_UNICODE_PUNCT="$MOSES/scripts/tokenizer/replace-unicode-punctuation.perl"
NORM_PUNC="$MOSES/scripts/tokenizer/normalize-punctuation.perl"
REM_NON_PRINT_CHAR="$MOSES/scripts/tokenizer/remove-non-printing-char.perl"
TOKENIZER="$MOSES/scripts/tokenizer/tokenizer.perl"
set -e
function clean () { \
  cat - | "$REPLACE_UNICODE_PUNCT" | "$NORM_PUNC" -l $1 | \
          "$REM_NON_PRINT_CHAR" | "$TOKENIZER" \
          -no-escape -threads $(grep -c ^processor /proc/cpuinfo) -l $1;}

# "$DATA/$L1_DATA" | clean "$L1" > "$DATA/${L1_DATA}.cleaned"
# "$DATA/$L2_DATA" | clean "$L2" > "$DATA/${L2_DATA}.cleaned"

# Usage example
head "$DATA/news.2017.et.shuffled" | clean en > "$DATA/news.2017.et.shuffled.cleaned" &

fatal: destination path '/home/hawk/Programming/Study/DS_Project/DS_Project/tools/mosesdecoder' already exists and is not an empty directory.
Tokenizer Version 1.1
Language: en
Number of threads: 8


### BPE codes generating

In [5]:
%%bash
FASTBPE="$TOOLS/fastBPE"
FAST="$FASTBPE/fast"
git clone https://github.com/glample/fastBPE "$FASTBPE"

g++ -std=c++11 -pthread -O3 "$FASTBPE/main.cc" -IfastBPE -o "$FAST"
"$FAST" learnbpe 40000 "$DATA/${L1_DATA}.cleaned" "$DATA/${L2_DATA}.cleaned" > "$DATA/BPE_codes"
"$FAST" applybpe "$DATA/${L1_DATA}.40000" "$DATA/${L1_DATA}.cleaned" "$DATA/BPE_codes"
"$FAST" applybpe "$DATA/${L2_DATA}.40000" "$DATA/${L2_DATA}.cleaned" "$DATA/BPE_codes"

"$FAST" getvocab "$DATA/${L1_DATA}.40000" > "$DATA/vocab.${L1_DATA}.40000" 
"$FAST" getvocab "$DATA/${L2_DATA}.40000" > "$DATA/vocab.${L2_DATA}.40000" 

# Add splitting data on train, valid and test

UsageError: Cell magic `%%shell` not found.


### N-gram Translation Table Inferring

In [ ]:
![ -f $RESOURCES/cc.cs.300.vec ] || wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.cs.300.vec.gz -P  "$RESOURCES"
![ -f $RESOURCES/cc.en.300.vec ] || wget -nc https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.vec.gz -P "$RESOURCES"
!gzip -d "$RESOURCES/cc.cs.300.vec.gz"
!gzip -d "$RESOURCES/cc.en.300.vec.gz"
!git clone https://github.com/artetxem/vecmap.git "$TOOLS"
!python3 "$TOOLS/map_embeddings.py" --unsupervised "$RESOURCES/cc.cs.300.vec" "$RESOURCES/cc.en.300.vec" "$RESOURCES/cs_mapped.vec" "$RESOURCES/en_mapped.vec" 

# TODO add the table inferring from above cross-lingual embeddings

## Model implementation

### Tools

In [ ]:
from torch import nn

#src https://pytorch.org/tutorials/beginner/transformer_tutorial.html
class PositionalEncoding(nn.Module):

    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)  

### BaseModel



Temporarily hyperparameters are hardcoded for better readability 


Sources:
1. https://pytorch.org/tutorials/beginner/transformer_tutorial.html
2. https://github.com/facebookresearch/XLM/blob/master/src/model/transformer.py
3. https://discuss.pytorch.org/t/memory-mask-in-nn-transformer/55230/5

In [ ]:
from torch.nn import TransformerDecoder, TransformerDecoderLayer,\\
                     TransformerEncoder, TransformerEncoderLayer

def get_masks(slen, lengths, causal): #[2]
    """
    Generate hidden states mask, and optionally an attention mask.
    """
    bs = lengths.size(0)
    alen = torch.arange(slen, dtype=torch.long, device=lengths.device)
    mask = alen < lengths[:, None]

    # attention mask is the same as mask, or triangular inferior attention (causal)
    if causal:
        attn_mask = alen[None, None, :].repeat(bs, slen, 1) <= alen[None, :, None]
    else:
        attn_mask = mask

    return mask, attn_mask


class Encoder(nn.Module):
    def __init__(self, voc_size, d_model, nlayers=6, nheads=8,):
        super(BaseModel, self).__init__()
        self.d_model = d_model
        self.embedding = nn.Embedding(voc_size, d_model)
        self.pos_encoder = PositionalEncoding(d_model)
        encoder_layers = TransformerEncoderLayer(d_model, nheads, dim_feedforward=1024, activation='gelu')
        self.encoder = TransformerEncoder(encoder_layers, nlayers)

        self.init_weights()

    def init_weights(self):
        initrange = 0.1
        self.embedding.init.normal_(self.embedding.weight, mean=0, std=self.d_model ** -0.5) #[2] L46

    def forward(self, src, src_lengths, trg):
        src_len, bs = src.size()

        src_mask, _ = get_masks(src, src_lengths, False) # TODO check causality parameter

        src = self.encoder(src)
        src = self.pos_encoder(src)
        output = self.encoder(src, src_mask)

        return output


class Decoder(nn.Module):
    def __init__(self, voc_size, d_model, nlayers=6, nheads=8,):
        super(BaseModel, self).__init__()
        self.d_model = d_model
        self.embedding = nn.Embedding(voc_size, d_model)
        self.pos_encoder = PositionalEncoding(d_model)
        encoder_layers = TransformerEncoderLayer(d_model, nheads, dim_feedforward=1024, activation='gelu')
        self.encoder = TransformerEncoder(encoder_layers, nlayers)

        self.init_weights()

    def init_weights(self):
        initrange = 0.1
        self.embedding.init.normal_(self.embedding.weight, mean=0, std=self.d_model ** -0.5) #[2] L46

    def forward(self, src, src_lengths, trg):
        src_len, bs = src.size()

        src_mask, _ = get_masks(src, src_lengths, False) # TODO check causality parameter

        src = self.encoder(src)
        src = self.pos_encoder(src)
        output = self.encoder(src, src_mask)

        return output







### Training

In [ ]:


def training(l1_training, l2_training, lambda):
    
    x, lengths, positions, langs = generate_batch(lang1, lang2, 'pred')
    

        params = self.params
        name = 'model' if params.encoder_only else 'encoder'
        model = getattr(self, name)
        model.train()

        # generate batch / select words to predict
        x, lengths, positions, langs, _ = self.generate_batch(lang1, lang2, 'pred')
        x, lengths, positions, langs, _ = self.round_batch(x, lengths, positions, langs)
        x, y, pred_mask = self.mask_out(x, lengths)

        # cuda
        x, y, pred_mask, lengths, positions, langs = to_cuda(x, y, pred_mask, lengths, positions, langs)

        # forward / loss
        tensor = model('fwd', x=x, lengths=lengths, positions=positions, langs=langs, causal=False)
        _, loss = model('predict', tensor=tensor, pred_mask=pred_mask, y=y, get_scores=False)
        self.stats[('MLM-%s' % lang1) if lang2 is None else ('MLM-%s-%s' % (lang1, lang2))].append(loss.item())
        loss = lambda_coeff * loss

        # optimize
        self.optimize(loss)

        # number of processed sentences / words
        self.n_sentences += params.batch_size
        self.stats['processed_s'] += lengths.size(0)
        self.stats['processed_w'] += pred_mask.sum().item()

# Archive


### Encoder/Decoder

In [ ]:
class Encoder(nn.Module):
  def __init__(self):
    super().__init__()

    self.src_word_emds = nn.Embedding(n_src_vocab, d_word_vec, padding_idx=pad_idx)
    self.position_enc = PositionalEncoding(d_word_vec, n_position=n_position)
    
class Decoder(nn.Module):
  def __init__(self):
    super().__init__()
    self.trg_word_emds = create_emb_layer()

### BaseModel

In [ ]:
class BaseModel(nn.Module):

    def __init__(self, voc_size, d_model, nlayers=6, nheads=8,):
        super(BaseModel, self).__init__()
        # TODELETE
        # self.src_mask = None
        self.d_model = d_model
        self.embedding = nn.Embedding(voc_size, d_model)
        self.pos_encoder = PositionalEncoding(d_model)
        encoder_layers = TransformerEncoderLayer(d_model, nheads, dim_feedforward=1024, activation='gelu')
        self.encoder = TransformerEncoder(encoder_layers, nlayers)


        # decoder_layers = TransformerDecoderLayer(d_model, nheads, dim_feedforward=1024, activation='gelu')
        # self.decoder = nn.TransformerDecoder(d_model, nlayers)

        self.init_weights()

    def _generate_square_subsequent_mask(self, sz): #[1]
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask

    def _generate_memory_mask(trg_sz, src_sz): #[3]
        mask = (torch.triu(torch.ones(src_sz, trg_sz)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask

    def init_weights(self):
        initrange = 0.1
        self.embedding.init.normal_(self.embedding.weight, mean=0, std=self.d_model ** -0.5) #[2] L46

    def forward(self, src, src_lengths, trg):
        # TODELETE
        # if self.src_mask is None or self.src_mask.size(0) != len(src): #[1]
        #     device = src.device
        #     self.src_mask = self._generate_square_subsequent_mask(len(src)).to(device)

        src_len, bs = src.size()

        src_mask, attn_mask = get_masks(src, src_lengths, False) # TODO check causality parameter

        # TODELETE
        src = self.encoder(src) #* math.sqrt(self.d_model)
        src = self.pos_encoder(src)
        output = self.encoder(src, src_mask)

        return output
